##### Cleaning dataset for training

In [ ]:
import pandas as pd
import os
import numpy as np
os.chdir('C:/Users/dalto/OneDrive/Pictures/Documents/Projects/Coding Projects/woba modeling/data/pitch/')

In [5]:
df = pd.read_csv('pitch_23_25_w.csv')
df1 = pd.read_csv('pitch_18_23_w.csv')
df2 = pd.read_csv('pitch_23_w.csv')
df3 = pd.read_csv('pitch_25_w.csv')

#### statcast name cleaning

In [6]:
import unicodedata
def clean_names(df, name_col):
    # Make a copy to avoid modifying the original DataFrame
    df = df.copy()
    
    # Convert to lowercase and remove leading/trailing whitespace
    df[name_col] = df[name_col].str.lower().str.strip()
    
    # Remove accents
    df[name_col] = df[name_col].apply(lambda x: ''.join(
        c for c in unicodedata.normalize('NFKD', str(x))
        if not unicodedata.combining(c)
    ))
    
    # Keep only letters and replace multiple spaces with single space
    df[name_col] = df[name_col].str.replace('[^a-z ]', '', regex=True).str.replace('\s+', ' ', regex=True)
    
    return df

<>:16: SyntaxWarning: invalid escape sequence '\s'
<>:16: SyntaxWarning: invalid escape sequence '\s'
C:\Users\dalto\AppData\Local\Temp\ipykernel_14296\2225678073.py:16: SyntaxWarning: invalid escape sequence '\s'
  df[name_col] = df[name_col].str.replace('[^a-z ]', '', regex=True).str.replace('\s+', ' ', regex=True)


##### merge

In [7]:
df = pd.concat([df, df1], ignore_index=True)
df = pd.concat([df, df2], ignore_index=True)
df = pd.concat([df, df3], ignore_index=True)

##### Spray Angle

In [8]:
spray_radian = np.arctan((df['hc_x'] - 125.42) / (198.27 - df['hc_y'])) # calcuate spray angle by setting plate as origin calcaluating angle from opp over adj
df['spray_angle'] = spray_radian * 180 / np.pi * .75

##### only select relevant features based on project aims

In [9]:
df['year'] = pd.to_datetime(df['game_date'])
df = df[['year', 'description', 'batter', 'woba_value', 'zone', 'b_stands', 'launch_speed', 'launch_angle', 'swing_path_tilt', 'attack_angle', 'attack_direction', 'spray_angle', 'bat_speed', 'swing_length', 'release_speed']]

##### map for catagorical feature

In [10]:
df = df[df['zone'].notna()]
df['zone'] = df['zone'].astype(int)

##### making it so pull is always positive

In [11]:
df.loc[df['b_stands'] == 'R', 'attack_direction'] *= -1
df.loc[df['b_stands'] == 'R', 'spray_angle'] *= -1

##### export

In [12]:
df.to_csv('pitch_cleaned.csv')